In [ ]:
# ====== Param vectors (self-contained) + mapping, MAC-Z exclus ======

def prepare_optimization_vectors():
    # defaults fallback (dacă nu au fost definite în alte celule)
    global indicator_params_default, short_params_default
    if "indicator_params_default" not in globals():
        indicator_params_default = {
            "fEma_Length": 61, "sEma_Length": 444,
            "ADX_len": 15, "ADX_smo": 10, "th": 5.47,
            "fastLength": 20, "slowLength": 43, "signalLength": 12,  # MACD clasic
            "MACD_options": "MACD",
            "BB_Length": 89, "BB_mult": 6.281,
            "sma_Length": 81, "volume_f": 0.87,
            "DClength": 79,
            "Sst": 0.10, "Sinc": 0.04, "Smax": 0.40,
            "bbMinWidth01": 9.3,
            "lengthz": 14, "lengthStdev": 16, "A": -0.1, "B": 0.5, "bbMinWidth02": 0.0
        }
    if "short_params_default" not in globals():
        short_params_default = {
            "tp": 3.6, "sl": 8.0,
            "atrPeriodSl": 50, "multiplierPeriodSl": 36.84, "trailOffset": 0.38,
            "reverse_exit": True, "exit_on_entry_loss": False,
            "Position": "Short"
        }

    # --- adăugăm fEma_Length, sEma_Length și ADX_smo în optimizare ---
    indicator_params_to_optimize = [
        "fEma_Length", "sEma_Length",
        "ADX_len", "ADX_smo", "th",
        "fastLength", "slowLength", "signalLength",
        "BB_Length", "BB_mult",
        "DClength"
    ]
    short_params_to_optimize = [
        "atrPeriodSl", "multiplierPeriodSl"
    ]

    # excludem MAC-Z din optimizare
    MACZ_KEYS = {"MACD_options", "lengthz", "lengthStdev", "A", "B"}
    names = [n for n in (indicator_params_to_optimize + short_params_to_optimize) if n not in MACZ_KEYS]

    # categorii (rotunjiri / bounds)
    LENGTH_INTS = {
        "fEma_Length", "sEma_Length", "ADX_len", "ADX_smo",
        "fastLength", "slowLength", "signalLength",
        "BB_Length", "sma_Length", "DClength", "atrPeriodSl"
    }
    SAR_FLOATS = {"Sst", "Sinc", "Smax"}
    PCT_FLOATS = {"tp", "sl", "bbMinWidth01"}  # procente
    POS_FLOATS = {"BB_mult", "volume_f", "th", "multiplierPeriodSl"}  # >0
    SHORT_FLOATS = {"trailOffset"}  # >=0

    # extinderi UNILATERALE acolo unde s-au lovit capetele
    # multiplicatori relativi la default (lo_mult, hi_mult)
    BOUND_OVERRIDES = {
        # ADX – ai lovit capătul inferior la len; coborâm și mai jos
        "ADX_len": (0.48, 1.17),              # 21 -> [~10, 25]
        "ADX_smo": (0.40, 1.40),              # 14 -> [~6, 20]  (validat oricum cu ADX_smo ≤ ADX_len)

        # prag ADX
        "th":      (0.33, 2.50),            

        # EMA – ușor mai sus la fast; slow rămâne cum e (nu era la capăt)
        "fEma_Length": (0.83, 1.30),          # 65 -> [54, 84]
        # "sEma_Length": (0.83, 1.17),        # lăsăm default (nu era la capăt)

        # BB – ai lovit capetele superioare, lărgim în sus
        "BB_Length": (0.83, 1.75),            # 51 -> [42, ~89]
        "BB_mult":  (0.50, 2.10),             # 3.0 -> [1.5, 6.3]

        # Donchian – ok cum e; îl lăsăm default
        # "DClength": (0.83, 1.17),

        # SL ATR – ai lovit capetele superioare, lărgim în sus
        "atrPeriodSl": (0.50, 2.00),          # 25 -> [12, 50]
        "multiplierPeriodSl": (0.50, 2.60),   # 14.25 -> [7.12, ~37.05]
    }


    start_params, bounds = [], []
    for name in names:
        val = float(indicator_params_default.get(name, short_params_default.get(name)))
        if name in LENGTH_INTS:
            lo_mult, hi_mult = BOUND_OVERRIDES.get(name, (0.83, 1.17))
            lo = max(2, int(round(val * lo_mult)))
            hi = max(lo + 1, int(round(val * hi_mult)))
            start_params.append(int(round(val)))
            bounds.append((lo, hi))
        elif name in SAR_FLOATS:
            lo = 0.01 if name == "Sst" else 0.02
            hi = 1.0 if name == "Smax" else 0.5
            start_params.append(val)
            bounds.append((lo, hi))
        elif name in PCT_FLOATS or name in POS_FLOATS or name in SHORT_FLOATS:
            lo_mult, hi_mult = BOUND_OVERRIDES.get(name, (0.50, 1.50))
            lo = max(0.0, val * lo_mult)
            hi = val * hi_mult
            start_params.append(val)
            bounds.append((lo, hi))
        else:
            continue

    return names, tuple(start_params), tuple(bounds)


def vector_to_param_dicts(x, names):
    """Mapează vectorul x -> (indicator_params, short_params), cu rotunjiri corecte, MACD clasic și cuantizare pe continui."""
    indicator_params = indicator_params_default.copy()
    short_params = short_params_default.copy()

    int_params = {
        "fEma_Length", "sEma_Length", "ADX_len", "ADX_smo",
        "fastLength", "slowLength", "signalLength",
        "sma_Length", "BB_Length", "DClength", "atrPeriodSl"
    }
    # pas minim pe câțiva continui (evită micro-rafinări fără impact material)
    QUANTIZE = {
        "th": 0.1,
        "BB_mult": 0.1,
        "multiplierPeriodSl": 0.25
    }

    for name, v in zip(names, x):
        if name in int_params:
            val = max(1, int(round(v)))
        else:
            val = float(v)
            if name in QUANTIZE:
                step = QUANTIZE[name]
                val = round(val / step) * step

        if name in indicator_params:
            indicator_params[name] = val
        elif name in short_params:
            short_params[name] = val

    # forțăm MACD clasic (nu MAC-Z)
    indicator_params["MACD_options"] = "MACD"
    for k in ("lengthz", "lengthStdev", "A", "B"):
        indicator_params.pop(k, None)

    return indicator_params, short_params



import configparser
import os
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
from dataclasses import dataclass
from pathlib import Path
from typing import Any, Dict, List, Optional

import numpy as np
import pandas as pd
from scipy.optimize import minimize

DEFAULT_DATA_DIR = os.environ.get("SUPER8_DATA_DIR", "Simboluri_Binance")
SYMBOLS_FILE = os.environ.get("SUPER8_SYMBOLS_FILE")
BINANCE_CFG_PATH = os.environ.get("BINANCE_CFG_PATH", "Binance.cfg")
RESULTS_PATH_DEFAULT = os.environ.get("SUPER8_RESULTS_PATH", "rezultate_optimizare.csv")


def _make_absolute(path: Path) -> Path:
    path = path.expanduser()
    if not path.is_absolute():
        path = (Path.cwd() / path).resolve()
    return path


DATA_DIR = _make_absolute(Path(DEFAULT_DATA_DIR))


def load_binance_credentials(cfg_path: Optional[str] = None, required: bool = False) -> Dict[str, str]:
    """Citește Binance.cfg și întoarce cheile API (dacă există)."""
    cfg_file = _make_absolute(Path(cfg_path or BINANCE_CFG_PATH))
    if not cfg_file.exists():
        if required:
            raise FileNotFoundError(f"Nu am găsit fișierul de configurare Binance.cfg la {cfg_file}.")
        print(f"[Avertisment] Nu am găsit Binance.cfg la {cfg_file}. Rularea backtest-ului nu necesită API key.")
        return {}

    parser = configparser.ConfigParser()
    parser.read(cfg_file, encoding='utf-8')

    section = None
    for candidate in ("binance", "BINANCE", "default", "DEFAULT"):
        if candidate in parser:
            section = parser[candidate]
            break

    if section is None:
        raise KeyError("Binance.cfg trebuie să conțină o secțiune 'binance'.")

    api_key = section.get('api_key') or section.get('API_KEY') or section.get('key')
    api_secret = section.get('api_secret') or section.get('API_SECRET') or section.get('secret')

    if required and (not api_key or not api_secret):
        raise ValueError("Binance.cfg nu conține câmpurile necesare 'api_key' și 'api_secret'.")

    if api_key or api_secret:
        masked = (api_key[:6] + '***') if api_key else '<nedisponibil>'
        print(f"[Info] Config Binance.cfg încărcat (cheie începe cu: {masked}).")

    return {'api_key': api_key or '', 'api_secret': api_secret or ''}


def list_available_symbols(data_dir: Optional[Path] = None) -> List[str]:
    directory = _make_absolute(Path(data_dir or DATA_DIR))
    if not directory.exists():
        raise FileNotFoundError(f"Folderul cu simboluri nu există: {directory}")
    return sorted({p.stem.upper() for p in directory.glob('*.csv')})


def load_symbol_universe(symbols_file: Optional[str] = SYMBOLS_FILE, data_dir: Optional[Path] = None) -> List[str]:
    if symbols_file:
        file_path = _make_absolute(Path(symbols_file))
        if not file_path.exists():
            raise FileNotFoundError(f"Nu am găsit fișierul cu simboluri: {file_path}")
        if file_path.suffix.lower() in {'.csv', '.tsv'}:
            df = pd.read_csv(file_path)
            for column in ('Symbol', 'symbol', 'SYMBOL'):
                if column in df.columns:
                    symbols = [str(v).strip().upper() for v in df[column] if str(v).strip()]
                    if symbols:
                        return symbols
        symbols = [line.strip().upper() for line in file_path.read_text(encoding='utf-8').splitlines() if line.strip()]
        if symbols:
            return symbols
        raise ValueError(f"Fișierul cu simboluri {file_path} nu conține valori valide.")
    return list_available_symbols(data_dir)


def save_results(df: pd.DataFrame, output_path: Optional[str] = None) -> Path:
    if df is None or df.empty:
        raise ValueError("Nu există date în results_df pentru a fi salvate.")
    target = _make_absolute(Path(output_path or RESULTS_PATH_DEFAULT))
    target.parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(target, index=False)
    print(f"[Info] Rezultatele au fost salvate în {target}")
    return target



def symbol_csv_path(symbol: str) -> Path:
    """Returnează calea completă către fișierul CSV aferent simbolului (în DATA_DIR)."""
    safe_symbol = symbol.upper().replace('/', '_')
    return (DATA_DIR / f"{safe_symbol}.csv").resolve()

def symbol_csv_path_case_insensitive(symbol: str) -> Optional[Path]:
    """Returnează calea existentă pentru simbol (ignorând extensia .csv/.CSV)."""
    base_name = symbol.upper().replace('/', '_')
    candidates = [DATA_DIR / f"{base_name}.csv", DATA_DIR / f"{base_name}.CSV"]
    for candidate in candidates:
        if candidate.exists():
            return candidate.resolve()
    return None

def _crossover(a: pd.Series, b: pd.Series) -> pd.Series:
    a1 = a.shift(1); b1 = b.shift(1)
    return (a1 <= b1) & (a > b)

def _crossunder(a: pd.Series, b: pd.Series) -> pd.Series:
    a1 = a.shift(1); b1 = b.shift(1)
    return (a1 >= b1) & (a < b)


# --- Helper functions for Price/Spread/Validation ---
def _rma(series: pd.Series, period: int) -> pd.Series:
    """Wilder's RMA (EMA with alpha = 1/period)."""
    alpha = 1.0 / period
    return series.ewm(alpha=alpha, adjust=False).mean()

def _ensure_price(df: pd.DataFrame) -> pd.DataFrame:
    """Ensure 'Price' column exists in df (use 'close' if present)."""
    if "Price" not in df.columns:
        if "close" in df.columns:
            df["Price"] = df["close"]
        else:
            raise KeyError("The CSV must contain a 'Price' or 'close' column.")
    for col in ("high", "low", "volume"):
        if col not in df.columns:
            raise KeyError(f"The CSV must contain column '{col}'.")
    return df

def _k_from_barlen(bar_length: str) -> float:
    bl = (bar_length or "").upper()
    if bl in ("M1", "1M"):  return 0.08
    if bl in ("M5", "5M"):  return 0.12
    if bl in ("M15", "15M"): return 0.18
    if bl in ("H2", "2H"):   return 0.26   # <- adaugă asta (valoare între H1 și H4)
    if bl in ("H4", "4H"):   return 0.30
    return 0.22  # default (H1)

def compute_spread(df: pd.DataFrame, pay_fees_with_bnb: bool = False, bar_length: str = "H1") -> float:
    """
    Compute per-side cost in log-return (Spread) and set df["Spread"] to that constant value.
    Spread = -ln(1 - (fee_pct + slip_pct)), where slip_pct = k * median((high-low)/Price) clipped [0.02%, 0.40%].
    Returns the numeric spread value.
    """
    df = _ensure_price(df)
    fee_pct = 0.0010 * (0.75 if pay_fees_with_bnb else 1.0)  # Spot fee: 0.10% or 0.075% with BNB
    rng = (df["high"] - df["low"]) / df["Price"]
    k = _k_from_barlen(bar_length)
    slip_pct = float(np.clip(k * float(np.nanmedian(rng)), 0.0002, 0.0040))
    spread_log = -np.log(1.0 - (fee_pct + slip_pct))
    df["Spread"] = spread_log
    return spread_log

def is_param_combo_valid(ind_params: Dict[str, Any], short_vals: Dict[str, Any]) -> bool:
    # EMA: fast EMA length < slow EMA length
    if int(ind_params["fEma_Length"]) >= int(ind_params["sEma_Length"]):
        return False
    # MACD: fast < slow
    if int(ind_params["fastLength"]) >= int(ind_params["slowLength"]):
        return False
    # ADX: smoothing ≤ length
    if int(ind_params.get("ADX_smo", ind_params["ADX_len"])) > int(ind_params["ADX_len"]):
        return False
    # SAR: toate pozitive; start/inc ≤ max
    if not (ind_params["Sst"] > 0 and ind_params["Sinc"] > 0 and ind_params["Smax"] > 0):
        return False
    if ind_params["Sst"] > ind_params["Smax"] or ind_params["Sinc"] > ind_params["Smax"]:
        return False
    # Bollinger Bands
    if int(ind_params["BB_Length"]) < 5:
        return False
    if float(ind_params["BB_mult"]) <= 0:
        return False
    # Donchian channel
    if int(ind_params["DClength"]) < 5:
        return False
    # Volume
    if int(ind_params["sma_Length"]) < 1:
        return False
    if float(ind_params["volume_f"]) <= 0:
        return False
    # Short params
    if int(short_vals["atrPeriodSl"]) < 1:
        return False
    if float(short_vals["multiplierPeriodSl"]) <= 0:
        return False
    if float(short_vals["trailOffset"]) < 0:
        return False
    if float(short_vals["tp"]) < 0 or float(short_vals["sl"]) < 0:
        return False
    return True


def why_invalid(ind_params, short_vals):
    reasons = []
    if int(ind_params["fEma_Length"]) >= int(ind_params["sEma_Length"]): reasons.append("fEma>=sEma")
    if int(ind_params["fastLength"])  >= int(ind_params["slowLength"]):  reasons.append("fast>=slow")
    if int(ind_params["ADX_smo"])     >  int(ind_params["ADX_len"]):     reasons.append("ADX_smo>ADX_len")
    if not (ind_params["Sst"]>0 and ind_params["Sinc"]>0 and ind_params["Smax"]>0): reasons.append("SAR<=0")
    if ind_params["Sst"]>ind_params["Smax"] or ind_params["Sinc"]>ind_params["Smax"]: reasons.append("SAR start/inc > max")
    if int(ind_params["BB_Length"])<5 or float(ind_params["BB_mult"])<=0: reasons.append("BB invalid")
    if int(ind_params["DClength"])<5: reasons.append("DC<5")
    if int(ind_params["sma_Length"])<1 or float(ind_params["volume_f"])<=0: reasons.append("volume invalid")
    if int(short_vals["atrPeriodSl"])<1 or float(short_vals["multiplierPeriodSl"])<=0 or float(short_vals["trailOffset"])<0:
        reasons.append("SL invalid")
    if float(short_vals["tp"])<0 or float(short_vals["sl"])<0: reasons.append("tp/sl<0")
    return reasons


class Super8Indicators:
    def __init__(self, params: Dict[str, Any]):
        self.p = params

    def _ema(self, s: pd.Series, span: int) -> pd.Series:
        return s.ewm(span=span, adjust=False).mean()

    def _adx_block(self, df: pd.DataFrame, length: int, smoothing: int, threshold: float) -> pd.DataFrame:
        # Calculate ADX, DI+ and DI-, plus long/short conditions
        tr = np.maximum(
            df["high"] - df["low"],
            np.maximum((df["high"] - df["Price"].shift(1)).abs(), (df["low"] - df["Price"].shift(1)).abs())
        )
        up = df["high"].diff()
        dn = -df["low"].diff()
        plus_dm = np.where((up > dn) & (up > 0), up, 0.0)
        minus_dm = np.where((dn > up) & (dn > 0), dn, 0.0)
        # Wilder's RMA for TR and directional movements
        tr_s = _rma(pd.Series(tr, index=df.index), length)
        plus_s = _rma(pd.Series(plus_dm, index=df.index), length)
        minus_s = _rma(pd.Series(minus_dm, index=df.index), length)
        plus_di = (plus_s / tr_s) * 100.0
        minus_di = (minus_s / tr_s) * 100.0
        denom = (plus_di + minus_di).clip(lower=1e-10)
        dx = ((plus_di - minus_di).abs() / denom) * 100.0
        adx = _rma(dx, smoothing)
        out = pd.DataFrame(index=df.index)
        out["adx"] = adx
        out["di_plus"] = plus_di
        out["di_minus"] = minus_di
        out["ADX_longCond"] = (plus_di > minus_di) & (adx > threshold)
        out["ADX_shortCond"] = (plus_di < minus_di) & (adx > threshold)
        return out

    def _sar_tv(self, high: pd.Series, low: pd.Series, start: float, step: float, smax: float, price: pd.Series) -> pd.Series:
        """Parabolic SAR (TradingView/Wilder style)."""
        h, l = high.to_numpy(), low.to_numpy()
        n = len(h)
        if n == 0:
            return pd.Series([], index=high.index, dtype=float)
        psar = np.zeros(n, dtype=float)
        # Determine initial trend direction (uptrend if price increases, else downtrend)
        up = True
        if n >= 2:
            if not (pd.isna(price.iloc[0]) or pd.isna(price.iloc[1])):
                up = bool(price.iloc[1] >= price.iloc[0])
            else:
                up = bool((h[1] + l[1]) >= (h[0] + l[0]))
        af = start
        ep = float(h[0] if up else l[0])
        psar[0] = float(l[0] if up else h[0])
        # Iterate to calculate SAR for each bar
        for i in range(1, n):
            psar[i] = psar[i-1] + af * (ep - psar[i-1])
            if up:
                # Clamping: SAR cannot go above last two lows
                if i >= 2:
                    psar[i] = min(psar[i], l[i-1], l[i-2])
                else:
                    psar[i] = min(psar[i], l[i-1])
                # New extreme point?
                if h[i] > ep:
                    ep = h[i]
                    af = min(af + step, smax)
                # Trend flip?
                if l[i] < psar[i]:
                    up = False
                    psar[i] = ep
                    ep = l[i]
                    af = start
            else:
                # Clamping: SAR cannot go below last two highs
                if i >= 2:
                    psar[i] = max(psar[i], h[i-1], h[i-2])
                else:
                    psar[i] = max(psar[i], h[i-1])
                # New extreme point?
                if l[i] < ep:
                    ep = l[i]
                    af = min(af + step, smax)
                # Trend flip?
                if h[i] > psar[i]:
                    up = True
                    psar[i] = ep
                    ep = h[i]
                    af = start
        return pd.Series(psar, index=high.index, name="SAR")

    def _macd(self, s: pd.Series, fast: int, slow: int, signal: int) -> pd.DataFrame:
        fast_ma = s.ewm(span=fast, adjust=False).mean()
        slow_ma = s.ewm(span=slow, adjust=False).mean()
        macd_line = fast_ma - slow_ma
        signal_line = macd_line.ewm(span=signal, adjust=False).mean()
        hist = macd_line - signal_line
        out = pd.DataFrame(index=s.index)
        out["hist"] = hist
        out["lMACD"] = macd_line
        out["sMACD"] = signal_line
        return out

    def _macz(self, df: pd.DataFrame) -> pd.DataFrame:
        # MAC-Z indicator (used only if MACD_options = "MAC-Z")
        lengthz = self.p["lengthz"]
        lengthStdev = self.p["lengthStdev"]
        A = self.p["A"]
        B = self.p["B"]
        signalLength = self.p["signalLength"]
        vol = df["volume"]
        px = df["Price"]
        vw_mean = (vol * px).rolling(window=lengthz, min_periods=lengthz).sum() / vol.rolling(window=lengthz, min_periods=lengthz).sum()
        vw_sd = (px - vw_mean).pow(2).rolling(window=lengthz, min_periods=lengthz).mean().pow(0.5)
        zscore = (px - vw_mean) / vw_sd
        macd_std = px.rolling(window=lengthStdev, min_periods=lengthStdev).std(ddof=0)
        macd = self._macd(px, int(self.p["fastLength"]), int(self.p["slowLength"]), int(self.p["signalLength"]))
        macz_line = (zscore * A) + (macd["lMACD"] / (macd_std * B))
        signal = macz_line.rolling(window=signalLength, min_periods=signalLength).mean()
        histmacz = macz_line - signal
        return pd.DataFrame({"histmacz": histmacz}, index=df.index)

    def compute(self, df: pd.DataFrame) -> pd.DataFrame:
        """Compute all technical indicators and entry/exit conditions needed for the strategy."""
        df = _ensure_price(df)
        p = self.p
        out = pd.DataFrame(index=df.index)

        # EMA long and short + conditions
        sEMA = self._ema(df["Price"], int(p["sEma_Length"]))
        fEMA = self._ema(df["Price"], int(p["fEma_Length"]))
        out["EMA_longCond"] = (fEMA > sEMA) & (sEMA > sEMA.shift(1))
        out["EMA_shortCond"] = (fEMA < sEMA) & (sEMA < sEMA.shift(1))

        # ADX indicator + conditions
        adx_df = self._adx_block(df, int(p["ADX_len"]), int(p.get("ADX_smo", p["ADX_len"])), float(p["th"]))
        out = out.join(adx_df)

        # Parabolic SAR + conditions
        sar = self._sar_tv(df["high"], df["low"], float(p["Sst"]), float(p["Sinc"]), float(p["Smax"]), df["Price"])
        out["SAR"] = sar
        out["SAR_longCond"] = (sar < df["Price"])
        out["SAR_shortCond"] = (sar > df["Price"])

        # MACD sau MAC-Z + conditions
        macd_df = self._macd(df["Price"], int(p["fastLength"]), int(p["slowLength"]), int(p["signalLength"]))
        out = out.join(macd_df)
        if p.get("MACD_options", "MACD") == "MAC-Z":
            out = out.join(self._macz(df))
        else:
            out["histmacz"] = out["hist"]
        out["MACD_longCond"] = out["histmacz"] > 0
        out["MACD_shortCond"] = out["histmacz"] < 0

        # Bollinger Bands (BB) + band width
        L = int(p["BB_Length"])
        mult = float(p["BB_mult"])
        mid = df["Price"].rolling(window=L, min_periods=L).mean()
        std = df["Price"].rolling(window=L, min_periods=L).std(ddof=0)
        bb_upper = mid + mult * std
        bb_lower = mid - mult * std
        out["BB_upper"] = bb_upper
        out["BB_lower"] = bb_lower
        out["BB_middle"] = mid
        out["BB_width"] = (bb_upper - bb_lower) / mid  # lățime normalizată

        # Volume (condiție volum ridicat)
        vol_sma = df["volume"].rolling(window=int(p["sma_Length"]), min_periods=1).mean()
        vol_flag = df["volume"] > vol_sma * float(p["volume_f"])
        out["VOL_shortCond"] = vol_flag
        out["VOL_longCond"] = vol_flag

        # Praguri minime BB width (convertite din procentaj la fracție)
        out["bbMinWidth01"] = float(p["bbMinWidth01"]) / 100.0
        out["bbMinWidth02"] = float(p["bbMinWidth02"]) / 100.0

        return out

@dataclass
class ShortParams:
    Position: str = "Both"
    TP_options: str = "Both"
    SL_options: str = "Both"
    tp: float = 3.6
    sl: float = 8.0
    atrPeriodSl: int = 21
    multiplierPeriodSl: float = 9.5
    trailOffset: float = 0.38
    reverse_exit: bool = True
    ignore_additional_entries: bool = True
    exit_on_entry_loss: bool = False
    start_time: Optional[pd.Timestamp] = None
    intrabar_touch: bool = True
    bar_path: str = "OLHC"
    no_same_bar_exit: bool = True

class Super8ShortBacktester:
    def __init__(self, indicator_params: Dict[str, Any], short_params: ShortParams, use_spread: bool = True, log_return: bool = True):
        self.indicator_params = indicator_params
        self.short_params = short_params
        self.use_spread = use_spread
        self.log_return = log_return
        self._last_bt = None  # store last backtest DataFrame if needed

    def run(self, df: pd.DataFrame) -> pd.DataFrame:
        # 1) Pregătire date
        data = df.copy()
        if "time" in data.columns:
            data["time"] = pd.to_datetime(data["time"], utc=True, errors="coerce")
            data = data.dropna(subset=["time"]).set_index("time").sort_index()

        data = _ensure_price(data)
        for col in ["high", "low", "volume"]:
            if col not in data.columns:
                raise ValueError(f"Data missing required column '{col}'.")

        if "Return" not in data.columns:
            data["Return"] = np.log(data["Price"] / data["Price"].shift(1))

        if "Spread" not in data.columns:
            raise ValueError("Missing 'Spread' in data — compute Spread before backtesting.")

        # 2) Indicatori
        indicators = Super8Indicators(self.indicator_params).compute(data)
        base = data.join(indicators, how="left")

        # 3) Condiții clasice
        EMA_shortCond = base.get("EMA_shortCond", pd.Series(False, index=base.index)).fillna(False)
        EMA_longCond  = base.get("EMA_longCond",  pd.Series(False, index=base.index)).fillna(False)
        ADX_shortCond = base.get("ADX_shortCond", pd.Series(False, index=base.index)).fillna(False)
        ADX_longCond  = base.get("ADX_longCond",  pd.Series(False, index=base.index)).fillna(False)
        SAR_shortCond = base.get("SAR_shortCond", pd.Series(False, index=base.index)).fillna(False)
        SAR_longCond  = base.get("SAR_longCond",  pd.Series(False, index=base.index)).fillna(False)
        VOL_shortCond = base.get("VOL_shortCond", pd.Series(False, index=base.index)).fillna(False)
        VOL_longCond  = base.get("VOL_longCond",  pd.Series(False, index=base.index)).fillna(False)

        # Lock: folosim doar MACD clasic
        h = base.get("hist", pd.Series(0.0, index=base.index))

        MACD_shortCond = h.lt(0).fillna(False)
        MACD_longCond  = h.gt(0).fillna(False)

        # Bollinger
        BB_upper  = base.get("BB_upper",  pd.Series(np.nan, index=base.index))
        BB_lower  = base.get("BB_lower",  pd.Series(np.nan, index=base.index))
        BB_middle = base.get("BB_middle", pd.Series(np.nan, index=base.index))
        BB_width  = base.get("BB_width",  pd.Series(np.nan, index=base.index))
        bbMin01   = base.get("bbMinWidth01", pd.Series(0.05, index=base.index))  # procent → fracție

        # 4) ENTRY = shortCond OR BB_short01 
        allow_short = (self.short_params.Position != "Long")
        BB_short01 = (allow_short) & (~ADX_longCond) & _crossover(base["high"], BB_upper) \
                     & EMA_shortCond & (BB_width > bbMin01)
        BB_short01 = BB_short01.fillna(False)

        shortCond  = (allow_short) & EMA_shortCond & ADX_shortCond & SAR_shortCond & MACD_shortCond & VOL_shortCond
        entry_cond = (shortCond | BB_short01).fillna(False)

        # 5) EXIT = reverse-exit (condiții opuse) + TP/SL
        exit_cond = (EMA_longCond | ADX_longCond | SAR_longCond | MACD_longCond).fillna(False)
        
        entry_sig = entry_cond.astype("boolean").shift(1).fillna(False).astype(bool)
        exit_sig  = exit_cond.astype("boolean").fillna(False).astype(bool)

        
        base["DBG_exit_cond"]     = exit_cond.astype(int)
        base["DBG_EMA_longCond"]  = EMA_longCond.astype(int)
        base["DBG_SAR_longCond"]  = SAR_longCond.astype(int)
        base["DBG_MACD_longCond"] = MACD_longCond.astype(int)

        # 6) ATR (Wilder/RMA) + "stair-step" pentru SHORT
        tr = np.maximum(
            base["high"] - base["low"],
            np.maximum((base["high"] - base["Price"].shift(1)).abs(),
                       (base["low"]  - base["Price"].shift(1)).abs())
        )
        atr_rma = _rma(pd.Series(tr, index=base.index), self.short_params.atrPeriodSl)
        atr_sl_short_raw = base["high"] + atr_rma * self.short_params.multiplierPeriodSl

        _open = base["open"] if "open" in base.columns else base["Price"].shift(1).fillna(base["Price"])

        atr_sl_short_series = pd.Series(index=base.index, dtype=float)
        prev_stop = np.nan
        for i in range(len(base.index)):
            raw = float(atr_sl_short_raw.iloc[i]) if not pd.isna(atr_sl_short_raw.iloc[i]) else np.nan
            o   = float(_open.iloc[i]) if not pd.isna(_open.iloc[i]) else raw
            if i == 0 or np.isnan(prev_stop) or np.isnan(raw) or np.isnan(o):
                val = raw
            else:
                val = min(raw, prev_stop) if o < prev_stop else raw
            atr_sl_short_series.iloc[i] = val
            prev_stop = val
        base["DBG_ATR_SL_Short"] = atr_sl_short_series

        # 7) Donchian pentru TP (dacă e cazul)
        if self.short_params.TP_options in ("Both", "Donchian"):
            DClower = base["low"].rolling(window=self.indicator_params["DClength"],
                                          min_periods=self.indicator_params["DClength"]).min()
        else:
            DClower = pd.Series(np.nan, index=base.index)

        # 8) Bucla de backtest (single-position; ENTRY next-bar, TP/SL & reverse same-bar)
        position = []
        pos = 0
        entry_price = np.nan

        for t, row in base.iterrows():
            # respect start_time
            if self.short_params.start_time is not None and t < self.short_params.start_time:
                position.append(0)
                continue

            price = row["Price"]

            # 1) aplică intrarea semnalată pe bara anterioară (next-bar)
            if pos == 0 and entry_sig.loc[t]:
                pos = -1
                entry_price = price  # intrare la close-ul barei curente

            # 2) pregătește nivelurile TP/SL pentru starea curentă
            avg_price = entry_price if (pos == -1 and not np.isnan(entry_price)) else price

            # --- TP (short) ---
            if self.short_params.TP_options == "Both":
                dc_val = DClower.loc[t] if 'DClower' in locals() and t in DClower.index else np.nan
                tp_level = np.nanmin([dc_val if not np.isnan(dc_val) else np.inf,
                                      (1.0 - self.short_params.tp/100.0) * avg_price])
            elif self.short_params.TP_options == "Normal":
                tp_level = (1.0 - self.short_params.tp/100.0) * avg_price
            elif self.short_params.TP_options == "Donchian":
                dc_val = DClower.loc[t] if 'DClower' in locals() and t in DClower.index else np.nan
                tp_level = dc_val if not np.isnan(dc_val) else avg_price
            else:
                tp_level = np.nan

            # --- SL (short) ATR stair-step ---
            atr_sl_val = atr_sl_short_series.loc[t] if t in atr_sl_short_series.index else np.nan
            if self.short_params.SL_options == "Both":
                sl_level = max(atr_sl_val, (1.0 + self.short_params.sl/100.0) * avg_price)
            elif self.short_params.SL_options == "Normal":
                sl_level = (1.0 + self.short_params.sl/100.0) * avg_price
            elif self.short_params.SL_options == "ATR":
                sl_level = atr_sl_val
            else:
                sl_level = np.nan

            # 3) dacă suntem în poziție, evaluează ieșirile SAME-BAR (TP/SL și reverse)
            if pos == -1:
                tp_hit = (not np.isnan(tp_level)) and (price <= tp_level)
                sl_hit = (not np.isnan(sl_level)) and (price >= sl_level)
                rev_hit = bool(self.short_params.reverse_exit and exit_sig.loc[t])

                if tp_hit or sl_hit or rev_hit:
                    pos = 0
                    entry_price = np.nan

            # 4) abia acum salvezi poziția pentru bara curentă (P&L pe bară folosește această stare)
            position.append(pos)

        base["position"] = position



        # 9) Randamente & costuri
        base["Return"] = base["Return"].fillna(0.0)
        base["strategy"] = base["position"].fillna(0) * base["Return"]

        
        pos_changes = base["position"].astype(int).diff().fillna(base["position"]).abs()
        base["trades"] = np.where(pos_changes != 0, 1, 0)

        if self.use_spread:
            base["Spread"] = base["Spread"].fillna(0.0)
            base["strategy"] = base["strategy"] - base["trades"] * base["Spread"]

        # 10) Cumulative returns
        if self.log_return:
            base["creturn"]   = base["Return"].cumsum().apply(np.exp)
            base["cstrategy"] = base["strategy"].cumsum().apply(np.exp)
        else:
            base["creturn"]   = (1.0 + base["Return"]).cumprod()
            base["cstrategy"] = (1.0 + base["strategy"]).cumprod()

        self._last_bt = base.copy()
        return base


def _get_default(name: str) -> Any:
    # Retrieve default value from appropriate dict
    return indicator_params_default.get(name, short_params_default.get(name))


# Inițializăm vectorii de optimizare
names, start_params, bounds = prepare_optimization_vectors()
print("Optimizez", len(names), "parametri:", names)

# Global cache for data (avoid reloading CSVs repeatedly in threads)
all_data: Dict[str, pd.DataFrame] = {}

def optimal_strategy(params_tuple: tuple, symbol: str, start: str, end: str, bar_length: str = "H1") -> float:
    """
    Objective function for optimization: returns the NEGATIVE of strategy performance (final wealth).
    The optimizer will minimize this (so maximizing final performance).
    Skips invalid parameter combinations (returns a neutral value for those).
    """
    try:
        # Load data from cache or file
        if symbol in all_data:
            df = all_data[symbol]
        else:
            file_path = symbol_csv_path_case_insensitive(symbol) or symbol_csv_path(symbol)
            if not file_path.exists():
                return 1e6  # skip this trial if data not found
            df = pd.read_csv(file_path, index_col='time', parse_dates=True)
            df = _ensure_price(df)
            COST_PER_SIDE = -np.log(1.0 - 0.00055)
            df["Spread"] = COST_PER_SIDE
            all_data[symbol] = df

        # Filter backtest period
        if not end:
            df_period = df.loc[start:]
        else:
            df_period = df.loc[start: end]
        if df_period.empty:
            return 1e6

        # Construct parameter dicts from tuple
        ind_params = indicator_params_default.copy()
        short_vals = short_params_default.copy()
        param_list = list(params_tuple)
        for i, name in enumerate(names):
            val = param_list[i]
            if name in ind_params:
                if name in ["sEma_Length", "fEma_Length", "ADX_len", "ADX_smo",
                            "fastLength", "slowLength", "signalLength",
                            "sma_Length", "BB_Length", "DClength"]:
                    ind_params[name] = max(1, int(round(val)))
                else:
                    ind_params[name] = float(val)
            else:
                # short strategy params
                if name == "atrPeriodSl":
                    short_vals[name] = max(1, int(round(val)))
                else:
                    short_vals[name] = float(val)

        # Set non-optimized options to default
        ind_params["MACD_options"] = indicator_params_default["MACD_options"]
        ind_params["lengthz"] = indicator_params_default["lengthz"]
        ind_params["lengthStdev"] = indicator_params_default["lengthStdev"]
        ind_params["A"] = indicator_params_default["A"]
        ind_params["B"] = indicator_params_default["B"]
        ind_params["bbMinWidth02"] = indicator_params_default["bbMinWidth02"]

        # Skip invalid combinations
        if not is_param_combo_valid(ind_params, short_vals):
            return 1e6

        # Run backtest for this parameter set
        short_obj = ShortParams(
            Position="Both", TP_options="Both", SL_options="Both",
            tp=short_vals["tp"], sl=short_vals["sl"],
            atrPeriodSl=short_vals["atrPeriodSl"],
            multiplierPeriodSl=short_vals["multiplierPeriodSl"],
            trailOffset=short_vals["trailOffset"]
        )
        backtester = Super8ShortBacktester(
            indicator_params=ind_params,
            short_params=short_obj,
            use_spread=True,
            log_return=True
        )
        bt = backtester.run(df_period)
        eq  = bt["cstrategy"].astype(float)
        bh  = bt["creturn"].astype(float)
        perf = float(eq.iloc[-1])
        bhf  = float(bh.iloc[-1]) if len(bh) else 1.0

        alpha  = (perf / bhf) if bhf > 0 else 0.0
        r      = bt["strategy"].fillna(0.0).values
        mu     = float(np.mean(r))
        sd     = float(np.std(r, ddof=1)) if len(r) > 1 else 0.0
        periods_per_year = 365*12  # H2 aprox
        sharpe = (mu/sd*np.sqrt(periods_per_year)) if sd > 0 else 0.0

        # penalizează seturi cu prea multe/puține tranzacții
        pos = bt["position"].astype(int)
        trades = int(((pos != pos.shift(1)).fillna(pos!=0)).sum())
        T = len(bt)
        tr_rate = trades / max(T,1)
        pen_tr = 0.0
        pen_tr += 2.0 * max(0.0, tr_rate - 0.04)   # >4% bare cu trade
        pen_tr += 2.0 * max(0.0, 0.01 - tr_rate)   # <1% bare cu trade

        score = (alpha * max(0.0, sharpe)) - pen_tr
        return -score


    except Exception:
        # On any error, return neutral value (so optimizer ignores/improves it)
        return -1.0

def optimize_strategy_parallel(symbol, start_par, bnds, start, end, bar_length, max_workers=4, seed=None):
    import numpy as np
    from concurrent.futures import ThreadPoolExecutor, as_completed
    from scipy.optimize import minimize

    rng = np.random.default_rng(seed)
    def rand_start():
        return tuple(rng.uniform(lo, hi) for (lo, hi) in bnds)

    best = None
    starts = [start_par] + [rand_start() for _ in range(max_workers - 1)]

    with ThreadPoolExecutor(max_workers=max_workers) as ex:
        futs = [ex.submit(
            minimize, optimal_strategy, s,
            args=(symbol, start, end or '', bar_length),
            method="Powell", bounds=bnds,
            options={"maxiter": 220, "xtol": 1e-2, "ftol": 1e-2}
        ) for s in starts]

        for f in as_completed(futs):
            try:
                r = f.result()
                if r.success and (best is None or r.fun < best.fun):
                    best = r
            except Exception as e:
                print("Eroare în calcul:", e)
    return best



def manage_symbol_data(symbol: str, file_path: Path) -> None:
    """Verifică existența fișierului local pentru simbol."""
    resolved = file_path.resolve()
    if resolved.exists():
        return
    alt_path = symbol_csv_path_case_insensitive(symbol)
    if alt_path is not None:
        return
    raise FileNotFoundError(
        f"Fișierul de date pentru {symbol} nu există: '{resolved}'.\n"
        f"Verifică DATA_DIR={DATA_DIR} sau setează SUPER8_DATA_DIR către folderul cu CSV-uri."
    )

def main(
    symbols: Optional[List[str]] = None,
    output_path: Optional[str] = None,
    cfg_path: Optional[str] = None,
    require_credentials: bool = False,
) -> pd.DataFrame:
    """Rulează optimizarea pe lista de simboluri indicată."""
    start_date = "2021-06-01"

    if symbols is None:
        try:
            symbols = load_symbol_universe(data_dir=DATA_DIR)
        except Exception as exc:
            raise RuntimeError(
                "Nu am reușit să determin lista de simboluri. Configurează SUPER8_SYMBOLS_FILE sau folderul cu CSV-uri."
            ) from exc
    if not symbols:
        raise ValueError("Lista de simboluri este goală.")

    load_binance_credentials(cfg_path, required=require_credentials)

    df_results = pd.DataFrame(columns=["Symbol", "Parameters", "Performance", "Trades"])

    for symbol in symbols:
        file_path = symbol_csv_path(symbol)
        try:
            manage_symbol_data(symbol, file_path)
            resolved_path = symbol_csv_path_case_insensitive(symbol) or file_path
        except Exception as e:
            print(f"Nu s-au putut obține datele pentru {symbol}: {e}")
            continue

        try:
            print(f"[{symbol}] Citim datele din: {resolved_path}")
            data = pd.read_csv(resolved_path, index_col='time', parse_dates=True)
        except Exception as e:
            print(f"Eroare la citirea fișierului pentru {symbol}: {e}")
            continue

        try:
            data = _ensure_price(data)
            COST_PER_SIDE = -np.log(1.0 - 0.00055)   # ≈ 0.00055015 log-return
            data["Spread"] = COST_PER_SIDE
        except Exception as e:
            print(f"[{symbol}] Eroare la pregătirea datelor (Price/Spread): {e}")
            continue

        all_data[symbol] = data

        last_idx = data.index.max()
        last_date_str = last_idx.strftime("%Y-%m-%d") if pd.notna(last_idx) else ""
        attempt = 0
        max_attempts = 4
        success = False
        last_exception = None

        while attempt < max_attempts and not success:
            try:
                print(f"Optimizare pentru simbolul {symbol} pe perioada {start_date} - {last_date_str}")
                opt_result = optimize_strategy_parallel(symbol, start_params, bounds, start_date, "", "H2")
                if opt_result is None or not opt_result.success:
                    raise ValueError("Optimizarea a eșuat.")

                best_params = opt_result.x
                output_params = []
                for i, name in enumerate(names):
                    val = best_params[i]
                    if name in ["sEma_Length", "fEma_Length", "ADX_len", "ADX_smo",
                                "fastLength", "slowLength", "signalLength",
                                "sma_Length", "BB_Length", "DClength", "atrPeriodSl"]:
                        v = max(1, int(round(val)))
                    else:
                        v = round(float(val), 3)
                    output_params.append(v)

                ind_params_opt = indicator_params_default.copy()
                short_vals_opt = short_params_default.copy()
                for i, name in enumerate(names):
                    if name in ind_params_opt:
                        if name in ["sEma_Length", "fEma_Length", "ADX_len", "ADX_smo",
                                    "fastLength", "slowLength", "signalLength",
                                    "sma_Length", "BB_Length", "DClength"]:
                            ind_params_opt[name] = max(1, int(round(best_params[i])))
                        else:
                            ind_params_opt[name] = float(best_params[i])
                    else:
                        if name == "atrPeriodSl":
                            short_vals_opt[name] = max(1, int(round(best_params[i])))
                        else:
                            short_vals_opt[name] = float(best_params[i])

                ind_params_opt["MACD_options"] = indicator_params_default["MACD_options"]
                ind_params_opt["lengthz"] = indicator_params_default["lengthz"]
                ind_params_opt["lengthStdev"] = indicator_params_default["lengthStdev"]
                ind_params_opt["A"] = indicator_params_default["A"]
                ind_params_opt["B"] = indicator_params_default["B"]
                ind_params_opt["bbMinWidth02"] = indicator_params_default["bbMinWidth02"]

                if not is_param_combo_valid(ind_params_opt, short_vals_opt):
                    print(f"Set de parametri invalid pentru {symbol}: {why_invalid(ind_params_opt, short_vals_opt)} (ignorat).")
                    break

                short_obj_opt = ShortParams(
                    Position="Both", TP_options="Both", SL_options="Both",
                    tp=short_vals_opt["tp"], sl=short_vals_opt["sl"],
                    atrPeriodSl=short_vals_opt["atrPeriodSl"],
                    multiplierPeriodSl=short_vals_opt["multiplierPeriodSl"],
                    trailOffset=short_vals_opt["trailOffset"]
                )
                backtester_opt = Super8ShortBacktester(indicator_params=ind_params_opt, short_params=short_obj_opt, use_spread=True, log_return=True)
                bt_full = backtester_opt.run(data.loc[start_date:])

                final_perf = float(bt_full["cstrategy"].iloc[-1]) if not bt_full.empty else 1.0
                pos_series = bt_full["position"]
                entry_events = (pos_series != 0) & (pos_series.shift(1).fillna(0) == 0)
                num_trades = int(entry_events.sum())

                new_row = {
                    "Symbol": symbol,
                    "Parameters": output_params,
                    "Performance": final_perf,
                    "Trades": num_trades
                }
                df_results = pd.concat([df_results, pd.DataFrame([new_row])], ignore_index=True)
                success = True

            except (ConnectionError, TimeoutError, OSError) as ex:
                last_exception = ex
                print(f"Eroare temporară la simbolul {symbol}: {ex}. Se reîncearcă.")
            except Exception as ex:
                last_exception = ex
                print(f"Eroare la simbolul {symbol}: {ex}. Trecem la următorul simbol.")
                break
            finally:
                attempt += 1
                if not success:
                    print(f"Încercare: {attempt}")
                    if attempt >= max_attempts:
                        print(f"Procesul pentru {symbol} a eșuat: număr maxim de încercări atins.")
                    elif isinstance(last_exception, (ConnectionError, TimeoutError, OSError)):
                        wait_time = 5 * attempt
                        print(f"Așteptăm {wait_time} secunde înainte de următoarea încercare.")
                        time.sleep(wait_time)

    if not df_results.empty:
        df_results = df_results.sort_values(by="Performance", ascending=False, ignore_index=True)

    if output_path:
        if df_results.empty:
            print("[Avertisment] Nu există rezultate pentru a fi salvate.")
        else:
            save_results(df_results, output_path)

    return df_results


# --- Exemplu de rulare a funcției main ---
if __name__ == "__main__":
    start_time = time.time()
    try:
        symbols = load_symbol_universe()
    except Exception as exc:
        raise SystemExit(f"[Eroare] Nu am putut construi lista de simboluri: {exc}") from exc

    results_df = main(
        symbols=symbols,
        cfg_path=BINANCE_CFG_PATH,
        output_path=RESULTS_PATH_DEFAULT,
    )

    end_time = time.time()
    print(f"Timpul total de execuție: {end_time - start_time:.2f} secunde")
    print(results_df)


In [ ]:
# Verificare rapidă a variabilelor de mediu și a căilor configurate
from pathlib import Path
import os

def verifica_configuratia_super8():
    campuri = {
        "SUPER8_DATA_DIR": os.getenv("SUPER8_DATA_DIR"),
        "BINANCE_CFG_PATH": os.getenv("BINANCE_CFG_PATH"),
        "SUPER8_RESULTS_PATH": os.getenv("SUPER8_RESULTS_PATH"),
        "SUPER8_SYMBOLS_FILE": os.getenv("SUPER8_SYMBOLS_FILE"),
    }
    rapoarte = []
    for cheie, valoare in campuri.items():
        if valoare:
            cale = Path(valoare).expanduser()
            rapoarte.append({
                "variabila": cheie,
                "valoare": valoare,
                "exista": cale.exists(),
                "tip": "fisier" if cale.is_file() else ("director" if cale.is_dir() else "intrare"),
            })
        else:
            rapoarte.append({
                "variabila": cheie,
                "valoare": None,
                "exista": False,
                "tip": "nedefinit"
            })
    return rapoarte

verifica_configuratia_super8()


In [ ]:
# Rulare backtest (poți ajusta lista de simboluri sau parametrul cfg_path)
symbol_list = load_symbol_universe()
results_df = main(symbols=symbol_list, cfg_path=BINANCE_CFG_PATH)
results_df


In [ ]:
# Salvare rezultate în CSV (schimbă output_path dacă dorești altă locație)
output_path = RESULTS_PATH_DEFAULT
save_path = save_results(results_df, output_path)
save_path
